In [26]:
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import matplotlib.pyplot as plt
import numpy as np
import warnings
warnings.filterwarnings("ignore")
import dash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output
import plotly.express as px
import plotly.figure_factory as ff
import json
import geojson
import requests
import plotly.graph_objs as go
import plotly.graph_objects as go

from plotly.subplots import make_subplots

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

#%matplotlib notebook
pd.options.display.max_columns = 150
pd.options.display.max_rows = 2000

data_root = '/Users/charlessimpson/Library/Mobile Documents/com~apple~CloudDocs/Documents/WorkspacePython/DATA/SQL/'

In [27]:
df2000=pd.read_csv("../csv/deces-2000.csv", sep=';', encoding='utf-8')
df2001=pd.read_csv("../csv/deces-2001.csv", sep=';', encoding='utf-8')
df2002=pd.read_csv("../csv/deces-2002.csv", sep=';', encoding='utf-8')
df2003=pd.read_csv("../csv/deces-2003.csv", sep=';', encoding='utf-8')
df2004=pd.read_csv("../csv/deces-2004.csv", sep=';', encoding='utf-8')
df2005=pd.read_csv("../csv/deces-2005.csv", sep=';', encoding='utf-8')
df2006=pd.read_csv("../csv/deces-2006.csv", sep=';', encoding='utf-8')
df2007=pd.read_csv("../csv/deces-2007.csv", sep=';', encoding='utf-8')
df2008=pd.read_csv("../csv/deces-2008.csv", sep=';', encoding='utf-8')
df2009=pd.read_csv("../csv/deces-2009.csv", sep=';', encoding='utf-8')
df2010=pd.read_csv("../csv/deces-2010.csv", sep=';', encoding='utf-8')
df2011=pd.read_csv("../csv/deces-2011.csv", sep=';', encoding='utf-8')
df2012=pd.read_csv("../csv/deces-2012.csv", sep=';', encoding='utf-8')
df2013=pd.read_csv("../csv/deces-2013.csv", sep=';', encoding='utf-8')
df2014=pd.read_csv("../csv/deces-2014.csv", sep=';', encoding='utf-8')
df2015=pd.read_csv("../csv/deces-2015.csv", sep=';', encoding='utf-8')
df2016=pd.read_csv("../csv/deces-2016.csv", sep=';', encoding='utf-8')
df2017=pd.read_csv("../csv/deces-2017.csv", sep=';', encoding='utf-8')
df2018=pd.read_csv("../csv/deces-2018.csv", sep=';', encoding='utf-8')
df2019=pd.read_csv("../csv/deces-2019.csv", sep=';', encoding='utf-8')
df2020 = pd.read_csv('../csv/deces_2020.csv', sep=';', encoding='utf-8')
#df2021=df2020
df_01_2021 = pd.read_csv('../csv/Deces_2021_M01.csv', sep=';', encoding='utf-8')
df_02_2021 = pd.read_csv('../csv/Deces_2021_M02.csv', sep=';', encoding='utf-8')
df_03_2021 = pd.read_csv('../csv/Deces_2021_M03.csv', sep=';', encoding='utf-8')
df_04_2021 = pd.read_csv('../csv/Deces_2021_M04.csv', sep=';', encoding='utf-8')
df_05_2021 = pd.read_csv('../csv/Deces_2021_M05.csv', sep=';', encoding='utf-8')
df_06_2021 = pd.read_csv('../csv/Deces_2021_M06.csv', sep=';', encoding='utf-8')
df_07_2021 = pd.read_csv('../csv/Deces_2021_M07.csv', sep=';', encoding='utf-8')
df_08_2021 = pd.read_csv('../csv/Deces_2021_M08.csv', sep=';', encoding='utf-8')
df_09_2021 = pd.read_csv('../csv/Deces_2021_M09.csv', sep=';', encoding='utf-8')
df_10_2021 = pd.read_csv('../csv/Deces_2021_M10.csv', sep=';', encoding='utf-8')
df_11_2021 = pd.read_csv('../csv/Deces_2021_M11.csv', sep=';', encoding='utf-8')
df_12_2021 = pd.read_csv('../csv/Deces_2021_M12.csv', sep=';', encoding='utf-8')


# Je concat toutes les BDD
data = pd.concat([df2000,df2001,df2002,df2003,df2004,df2005,df2006,df2007,df2008,df2009,df2010,df2011,df2012,df2013,df2014,df2015,df2016,df2017,df2018,df2019,df2020,df_01_2021,df_02_2021,df_03_2021,df_04_2021,df_05_2021,
                  df_06_2021,df_07_2021,df_08_2021,df_09_2021,df_10_2021,df_11_2021,df_12_2021])


In [41]:
# je récupère le numéro du département à 2 chiffres (attention aux 10 premiers départements)
def get_department(postal_code):
    postal_code = str(postal_code)
    return postal_code[:2]

# Renomage des colonnes
db = data.rename(columns={'nomprenom':'nom',
                        'datenaiss':'date_naiss',
                        'lieunaiss':'dpt_lieu_naiss',
                        'sexe':'sexe',
                        'commnaiss':'ville',
                        'paysnaiss':'pays',
                        'datedeces':'date_dc',
                        'lieudeces':'dpt_lieu_dc',
                        'actedeces':'dpt_lieu_deces'})

# je réduis ma BDD à la taille utile
#db = db[['nom','dpt_lieu_dc', 'date_naiss','date_dc']]

# je récupère les numéros de départements simplifiés pour les décès
db['dpt_deces'] = db['dpt_lieu_dc'].apply(lambda x : get_department(x))
print("Nombre de deces cumulés sur la période",db.shape)

# Nettoyage des numéros de départements pour s'assurer qu'on récupère correctement les 10 premiers départements
def non_int_filter(dpt_deces):
    try:
        return int(dpt_deces)
    except:
        return 0
db['dpt_deces'] = db['dpt_deces'].apply(lambda x : non_int_filter(x))
count_db = db.loc[db['dpt_deces']== 0]


# je récupère les numéros de départements simplifiés pour les naissance
db['dpt_lieu_naiss'] = db['dpt_lieu_naiss'].apply(lambda x : get_department(x))
db['dpt_lieu_naiss'] = db['dpt_lieu_naiss'].apply(lambda x : non_int_filter(x))
count_db = db.loc[db['dpt_lieu_naiss']== 0]


# J'identifie les décès sans département
#db = db.loc[db['dpt_deces']!= 0] # fonction qui supprimer les lignes sans départements
# transformer la data décès au bon format pour manipuler cette donnée 
db['date_deces']=pd.to_datetime((db.date_dc), format='%Y%m%d', errors='coerce')

# Nettoyage du format de la date de décès, pour avoir le mois et l'année au bon format
db=db.dropna(subset=['date_deces'])

def year(date_dc):
    date_dc = str(date_dc)
    return date_dc[:4]
db['year'] = db['date_dc'].apply(lambda x : year(x))

def month(date_dc):
    date_dc = str(date_dc)
    return date_dc[:6]
db['month'] = db['date_dc'].apply(lambda x : month(x))

#Suppression des DOMTOM
db=db[db['dpt_deces']!=97]
db=db[db['dpt_deces']!=98]
db=db[db['dpt_deces']!=99]

# Je rajoute la BDD des départements pour insérer les noms des départements
departements_info = pd.read_excel('../csv/departements-francais.xls')
number_to_departement = departements_info.set_index('NUMÉRO')['NOM'].to_dict()

# Nettoyage des déparmtents, j'enlève l'outre mer et je rajouter le nom des départements
number_to_departement[97] =  'Outre-mer'
db = db.rename(columns={'dpt_deces':'departement_number'})
db['departement'] = db['departement_number'].apply(lambda x : number_to_departement.get(x, ""))
wrong_dpt = db[db.departement_number.isin([98, 99])]
db = db[~db.departement_number.isin([98, 99])]

# je modifie les noms des départements qui ne sont pas correctes
db['departement'] = db['departement'].str.replace("Val-D'Oise", "Val-d'Oise")
db['departement'] = db['departement'].str.replace( 'Seine-St-Denis', "Seine-Saint-Denis")
db['departement'] = db['departement'].str.replace( "Côtes d'Armor", "Côtes-d'Armor")
db['departement'] = db['departement'].str.replace( "Vandée", "Vendée")


db.sample(3)

Nombre de deces cumulés sur la période (12792304, 10)


,nom,sexe,date_naiss,dpt_lieu_naiss,ville,pays,date_dc,dpt_lieu_dc,dpt_lieu_deces,departement_number,date_deces,year,month,departement
453823,VANLANCKER*MARIE LOUISE/,2,19071125,93,BONDY,NaN,20031002.0,45004,621,45,2003-10-02,2003,200310,Loiret
12269,SOUBROUILLARD*ROGER JULES/,1,19160312,57,PEVANGE,NaN,20080103.0,57132,000000001,57,2008-01-03,2008,200801,Moselle
408105,ABELLO*JEAN JOSEPH/,1,19301002,83,HYERES,NaN,20080923.0,83069,000000764,83,2008-09-23,2008,200809,Var


# Check database CPFM

In [50]:
db_month_db=db.groupby('year')['nom'].count().reset_index()
db_month_db.tail(5)

,year,nom
120,2017,593959
121,2018,596956
122,2019,599764
123,2020,654931
124,2021,632351


In [51]:
# je calcu l'age de décès du défunt
db['diff_in_D'] = abs(db['date_dc'] - db['date_naiss'])
db['diff_in_Y']=db['diff_in_D']/10000
db.sample(3)

,nom,sexe,date_naiss,dpt_lieu_naiss,ville,pays,date_dc,dpt_lieu_dc,dpt_lieu_deces,departement_number,date_deces,year,month,departement,diff_in_D,diff_in_Y
505837,RETIERE*PIERRE JULIEN ALEXIS/,1,19201130,44,SUCE SUR ERDRE,NaN,20001010.0,44201,28,44,2000-10-10,2000,200010,Loire-Atlantique,799880.0,79.9880
534041,LEFEUVRE*LUCIEN JEAN MARIE/,1,19220330,35,VAL D IZE,NaN,20081205.0,35353,49,35,2008-12-05,2008,200812,Ille-et-Vilaine,860875.0,86.0875
399924,BESNIER*ALBERT MARCEL/,1,19210116,53,SAINT-CALAIS-DU-DESERT,NaN,20010902.0,53147,223,53,2001-09-02,2001,200109,Mayenne,800786.0,80.0786


In [52]:
# je calcu l'age de décès du défunt
db=db[db['diff_in_Y']<=125]
db['diff_in_D'] = abs(db['date_dc'] - db['date_naiss'])
db['diff_in_Y']=db['diff_in_D']/10000
db.sort_values(by='diff_in_Y', ascending=False).head(2)

,nom,sexe,date_naiss,dpt_lieu_naiss,ville,pays,date_dc,dpt_lieu_dc,dpt_lieu_deces,departement_number,date_deces,year,month,departement,diff_in_D,diff_in_Y
529063,CARTOUX*THERESE/,2,18901006,84,VILLES-SUR-AUZON,NaN,20130502.0,84054,65,84,2013-05-02,2013,201305,Vaucluse,1229496.0,122.9496
187805,KERMICHE*MESSAOUDA/,2,18940000,93,DEPARTEMENT DE CONSTANTINE,NaN,20150416.0,26281,221,26,2015-04-16,2015,201504,Drôme,1210416.0,121.0416


In [56]:
# evolution du nombre de décès et l'âge moyen
db_age_moyen = db.groupby(['year'])['diff_in_Y'].agg(['count','mean']).reset_index()
db_age_moyen=db_age_moyen[db_age_moyen['year']>'2000']
db_age_moyen = db_age_moyen.rename(columns={'nomprenom':'nom',
                        'count':'Nombre de décès',
                        'mean':'Age moyen des défunts',
                       })
db_age_moyen.tail(5)

,year,Nombre de décès,Age moyen des défunts
120,2017,593959,79.291037
121,2018,596956,79.394618
122,2019,599764,79.575621
123,2020,654931,80.064368
124,2021,632351,79.805902


In [60]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Create figure with secondary y-axis
fig = make_subplots(specs=[[{"secondary_y": True}]])

# Add traces
fig.add_trace(
    go.Bar(x=db_age_moyen["year"], y=db_age_moyen["Nombre de décès"], name="Evolution du nombre de décès"),
    secondary_y=False, 
)

fig.add_trace(
    go.Scatter(x=db_age_moyen["year"], y=db_age_moyen["Age moyen des défunts"], name="Evolution de l'âge moyen"),
    secondary_y=True, 
)

# Add figure title
fig.update_layout(
    title_text="Evolution du nombre de décès par an et de l'âge moyen des défunts depuis 2000",
)

'''# Set x-axis title
fig.update_xaxes(title_text="% évolution décès par rapport à l'année précédente")'''

'''# Set y-axes titles
fig.update_yaxes(title_text="<b>Nombre décès par mois</b>", secondary_y=False)
fig.update_yaxes(title_text="<b>% par rapport à l'année précédente</b> ", secondary_y=True)'''
fig.write_html("cartes-inter/chart_evol_deces_2000_2020.html")

fig.show()

# Côte D'Or

In [64]:
db_cote_d_or=db[db['departement']=="Côte-d'Or"]
# evolution du nombre de décès et l'âge moyen
db_age_moyen_cote_d_or = db_cote_d_or.groupby(['year'])['diff_in_Y'].agg(['count','mean']).reset_index()
db_age_moyen_cote_d_or=db_age_moyen_cote_d_or[db_age_moyen_cote_d_or['year']>'2000']
db_age_moyen_cote_d_or = db_age_moyen_cote_d_or.rename(columns={'nomprenom':'nom',
                        'count':'Nombre de décès',
                        'mean':'Age moyen des défunts',
                       })
db_age_moyen_cote_d_or.tail(5)

,year,Nombre de décès,Age moyen des défunts
39,2017,5660,78.928141
40,2018,5542,79.008488
41,2019,5522,79.277611
42,2020,6263,79.753743
43,2021,5727,78.815596


In [68]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Create figure with secondary y-axis
fig = make_subplots(specs=[[{"secondary_y": True}]])

# Add traces
fig.add_trace(
    go.Bar(x=db_age_moyen_cote_d_or["year"], y=db_age_moyen_cote_d_or["Nombre de décès"], name="Evolution du nombre de décès"),
    secondary_y=False, 
)

fig.add_trace(
    go.Scatter(x=db_age_moyen_cote_d_or["year"], y=db_age_moyen_cote_d_or["Age moyen des défunts"], name="Evolution de l'âge moyen"),
    secondary_y=True, 
)

# Add figure title
fig.update_layout(
    title_text="Evolution du nombre de décès par an et de l'âge moyen des défunts depuis 2000 en Côte d'Or",
)

'''# Set x-axis title
fig.update_xaxes(title_text="% évolution décès par rapport à l'année précédente")'''

'''# Set y-axes titles
fig.update_yaxes(title_text="<b>Nombre décès par mois</b>", secondary_y=False)
fig.update_yaxes(title_text="<b>% par rapport à l'année précédente</b> ", secondary_y=True)'''
fig.write_html("cartes-inter/chart_evol_deces_2000_2020_cote_d_or.html")
fig.write_json("cartes-inter/chart_evol_deces_2000_2020_cote_d_or.json")

fig.show()